# Ⅰ.使用Selenium库爬取

In [ ]:
'''
功能：selenium库的简单使用
作者：吴小华
时间：2019-7-23 上午
'''
from selenium import webdriver
import time

#1.使用Chrome浏览器打开url
driver = webdriver.Chrome()
driver.get('http://www.autohome.com.cn')
time.sleep(5)

#2.向搜索框输入:路虎
text_button = driver.find_element_by_class_name('search-text')
text_button.send_keys('路虎')
time.sleep(2)

#3.点击搜索链接
click_button = driver.find_element_by_class_name('btn-search')
click_button.click()
time.sleep(5)

#4.输出搜索后的网页源代码
html = driver.page_source
print('结束')

In [ ]:
'''
功能：使用selenium库爬取邮电出版社书籍信息
作者：吴小华
时间：2019-7-23 上午
'''
from selenium import webdriver
from bs4 import BeautifulSoup
import time

#1.使用Chrome浏览器打开url
url = 'https://www.ptpress.com.cn/search/books'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

#2.向搜索框输入:python
test_button = driver.find_element_by_id('searchVal')
test_button.send_keys('python')
time.sleep(2)

#3.点击搜索链接
click_button = driver.find_element_by_css_selector('#app > div:nth-child(1) > div > div > div > button > i')
click_button.click()
time.sleep(2)

#4.输出搜索后的网页源代码
html = driver.page_source

#5.获取所有“python”的链接url
soup = BeautifulSoup(html,'html.parser')
soup.prettify
href_tag = soup.select('div.book_item > a')
child_url = []
for i in range(len(href_tag)):
    child_url.append('https://www.ptpress.com.cn' + href_tag[i].get('href'))

#6.进入url详情页，输出详情信息
driver = webdriver.Chrome()
for i in range(len(child_url)):
    print('-----------------------------')
    print('           第%d本书'%(i+1))
    print('-----------------------------')
    driver.get(child_url[i])
    time.sleep(2)
    
    child_html = driver.page_source
    child_soup = BeautifulSoup(child_html,'html.parser')
    child_soup.prettify
    
    info = child_soup.select('div.info > p')
    print('书名：' + info[0].get_text())
    print('作者：' + info[1].get_text())
    price = child_soup.select('span.price')
    print('价格：' + price[0].get_text())
    number = child_soup.select('div.key_attr > dl > dd')
    print('库存：' + number[0].get_text())
    #出版时间
    print_time = child_soup.select('table.bookBaseInfo > tbody > tr > td')
    print(print_time[4].get_text())
print('结束')

# Ⅱ.模拟登录

In [ ]:
'''
功能：使用表单模拟登录
作者：吴小华
时间：2019-7-23 下午
'''
import requests
from PIL import Image
from selenium import webdriver

sess = requests.Session()

#1.提取图片得到验证码
def pic_def():
    pic_url = 'http://www.tipdm.org/captcha.svl'
    pic = sess.get(pic_url)
    with open(r'C:\Users\苏哥哥\Desktop\111\pic.jpg','wb') as pf:
        pf.write(pic.content)
    ima = Image.open(r'C:\Users\苏哥哥\Desktop\111\pic.jpg')
    ima.show()
    imag_text = input('请输入验证码：')
    return imag_text

#2.模拟表单登录
url = 'http://www.tipdm.org/login.jspx'
login_data = {'username':'pc2019','password':'pc2019','captcha':pic_def()}
html = sess.post(url,data = login_data)
print(html.status_code)
print(html.url)

#3.打开登录成功后的页面
driver = webdriver.Chrome()
driver.get(html.url)

In [ ]:
'''
功能：使用手动成功登录后的临时Cookies模拟登录
作者：吴小华
时间：2019-7-23 下午
'''
import requests

#1.登录后的Cookies
cookies_str = '_site_id_cookie=1; clientlanguage=zh_CN; uniqueVisitorId=43417a73-6e3d-e8ab-8223-66f3c3e8fdbe; __qc_wId=963; pgv_pvid=5583736872; username=pc2019; JSESSIONID=CC60C25B344FE0C164D78B88200FA29F; JSESSIONID=CC60C25B344FE0C164D78B88200FA29F'
cookie = {}
for i in cookies_str.split(';'):
    key,value = i.split('=')
    cookie[key] = value
print(cookie)

#2.模拟Cookies登录
url = 'http://www.tipdm.org/login.jspx'
resp = requests.get(url,cookies = cookie)
print(resp.url)

In [ ]:
'''
功能：使用保存的Cookies模拟登录
作者：吴小华
时间：2019-7-23 下午
'''
import requests
from PIL import Image
from http import cookiejar

sess = requests.Session()
sess.cookies = cookiejar.LWPCookieJar('cookie')

#1.提取图片得到验证码
def pic_def():
    pic_url = 'http://www.tipdm.org/captcha.svl'
    pic = sess.get(pic_url)
    with open(r'C:\Users\苏哥哥\Desktop\111\pic.jpg','wb') as pf:
        pf.write(pic.content)
    ima = Image.open(r'C:\Users\苏哥哥\Desktop\111\pic.jpg')
    ima.show()
    imag_text = input('请输入验证码：')
    return imag_text

#2.模拟表单登录
url = 'http://www.tipdm.org/login.jspx'
login_data = {'username':'pc2019','password':'pc2019','captcha':pic_def()}
html = sess.post(url,data = login_data)
print(html.status_code)
print(html.url)

#3.保存Cookies
sess.cookies.save(ignore_discard = True,ignore_expires = True)
try:
    sess.cookies.load(ignore_discard = True)
except:
    print('没有加载成功...')

#4.模拟Cookies登录
cookie_html = sess.get(url)
print(cookie_html.url)